In [1]:
import pandas as pd
import csv
import networkx as nx
import numpy as np
import math
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import json
import os

## build the firm graph

In [2]:
'''edgelist of  firms with customer relation'''

customerEdges = []
customers = []
filepath = '/home/aswin/projects/econIIM/granShockAggrVolatility/data/linkages_US.txt'
with open(filepath, 'r') as file:
    for line in file:
        s = line.split()
        if s[2] == 'Customer':
            customerEdges.append([s[1],s[0]])
            customers.append(s[0])
            customers.append(s[1])
customers =  list(set(customers))

In [3]:
'''create SIC to NAICS conversion dict'''

cnvrsnTbl =pd.read_csv('../data/conversionTable.csv',encoding= 'utf-8', delimiter = ',') 
sic_naics_dict = dict(zip(cnvrsnTbl['SIC'], cnvrsnTbl['NAICS']))


In [4]:
len(sic_naics_dict)

1006

In [5]:
'''
The SIQ dataset containig
firms and it's sector information
'''


IQ = pd.read_csv('../data/description_US1.csv',encoding= 'utf-8', delimiter = ',', keep_default_na=False) #, error_bad_lines=False)#, warn_bad_lines=True,)

for i in range(len(IQ)):
    if IQ.iloc[i][1][0:2] != 'IQ':
        IQ.iloc[i][1] = IQ.iloc[i][2]
        IQ.iloc[i][2] = IQ.iloc[i][3]
        IQ.iloc[i][3] = IQ.iloc[i][4]
        IQ.iloc[i][4] = IQ.iloc[i][5]
        IQ.iloc[i][5] = IQ.iloc[i][6]
        

In [6]:
#make customer dataframe
customeriq = []
for i in range(len(customers)):
    new = 'IQ' + customers[i]
    if new not in customeriq:
        customeriq.append(new)


IQcustomer = IQ[IQ['ID'].isin(customeriq)]

In [7]:

''' create dict of the form:
{firm : NAICS code} '''

firmNAICS = {}
#missingSIC = []
#missingSICDescription = {}

for i  in range(len(IQcustomer)):
    sic = IQcustomer.iloc[i]['SIC']
    if sic != '0' and sic != 'nan'and sic in sic_naics_dict:
            firmNAICS[IQcustomer.iloc[i]['ID'][2:]] = str(sic_naics_dict[sic])
    
        



In [ ]:
len(firmNAICS)

In [ ]:
#IO table
io = pd.read_excel('../data/Use_SUT_Framework_2017_DET.xlsx',sheet_name = "2017",)
io = io.iloc[:394, :394]
io = io.fillna(0) 
io = io.div(io.sum(axis=0), axis=1) #normalization

In [ ]:
naicsINio = io.columns.tolist()

# ************************** detail <-> naics **************************

In [8]:
df = pd.read_excel('../data/Use_SUT_Framework_2017_DET.xlsx',sheet_name = "NAICS Codes",)


In [9]:
df = df.dropna(subset=['detail', 'naics'])

In [10]:
# Function to expand ranges like '11113-6'
def expand_naics_range(naics_code):
    expanded = []
    # Split by commas first
    for part in naics_code.split(','):
        part = part.strip()  # Remove any extra spaces
        if '-' in part:  # Check if there is a range
            base, end = part.split('-')
            start = int(base)
            end = int(base[:-len(str(end))] + end)  # Combine base and suffix to form end of range
            expanded.extend(map(str, range(start, end + 1)))
        else:
            expanded.append(part)
    return expanded

In [11]:
detailNAICSdict = {}

In [12]:
for i in range(len(df)):
    if type(df.iloc[i]['naics']) == int:
        detailNAICSdict[df.iloc[i]['detail']] = df.iloc[i]['naics']
    else:
        X = df.iloc[i]['naics'].split(',')
        if len(X) == 1:
            codes = expand_naics_range(X[0])
            #for code in codes:
            detailNAICSdict[df.iloc[i]['detail']] = codes
        else:
            lst = []
            for x in X:
                if '-' in x:
                    Y = expand_naics_range(X[0])
                    for y in Y:
                        lst.append(y)
                else:
                    lst.append(x)
            detailNAICSdict[df.iloc[i]['detail']] = lst
                

                
            
    

In [13]:

# Inverting the dictionary
naicsDetailDICT = {}

for detail, naics_values in detailNAICSdict.items():
    # Ensure values are treated as a list (if single integer, convert to a list)
    if not isinstance(naics_values, list):
        naics_values = [naics_values]

    # Add each value as key in the inverted dictionary, with detail as its value
    for value in naics_values:
        naicsDetailDICT[str(value)] = str(detail)



# ****************************************************************

In [14]:
'''
dicts: firm 2 naics
        naics 2 detail(sector)

make: naics 2 detail dict

'''
firmDetail = {}
missing = []

# Iterate over firmNAICS to create firmDetail
for firm, naics in firmNAICS.items():
    # Check if the firm's naics is present in naicsDetailDICT
    if naics in naicsDetailDICT:
        # Assign the detail to the firm
        firmDetail[firm] = naicsDetailDICT[naics]
    
    elif naics[0:5] in naicsDetailDICT:
        firmDetail[firm] = naicsDetailDICT[naics[0:5]]
    elif naics[0:4] in naicsDetailDICT:
        firmDetail[firm] = naicsDetailDICT[naics[0:4]]
    elif naics[0:3] in naicsDetailDICT:
        firmDetail[firm] = naicsDetailDICT[naics[0:3]]

    else:
        missing.append(naics)



In [ ]:
firmDetail

In [ ]:
''' function to convert the io table
to a weighted edge list'''

def io2EdgLst(df):
    edge = []
    cols  = df.columns
    for i in range(0,len(df)):
        for j in range(0,len(df)):
            
            if not math.isnan(list(io.iloc[i])[j]):
                edge.append([cols[i],cols[j],list(io.iloc[i])[j]])

            
    return edge

In [ ]:
sectorEdges =  io2EdgLst(io)

In [43]:
G = nx.DiGraph()
G.add_edges_from(customerEdges)

In [45]:
# Assuming G is your directed graph
# Get the largest weakly connected component
largest_wcc = max(nx.weakly_connected_components(G), key=len)

# Create a subgraph from the largest WCC
G = G.subgraph(largest_wcc).copy()

In [46]:
print(G)

DiGraph with 78020 nodes and 175130 edges


In [47]:
detailCode = list(set(firmDetail.values()))
detailSerialMapping = {code:i for i, code in enumerate(detailCode)}
firmSerial = {firm: detailSerialMapping[detail] for firm, detail in firmDetail.items()}


In [48]:
nx.set_node_attributes(G, firmSerial, 'label')


In [49]:
#G = nx.read_graphml("FirmGraphUNweightedSctrLabled.graphml")
node_mapping = {node: int(idx) for idx, node in enumerate(G.nodes())}

# Relabel the graph using the mapping
G = nx.relabel_nodes(G, node_mapping)

In [54]:




# Print or visualize the GCC
print(f"The number of nodes in the GCC (WCC) is: {len(G.nodes())}")
print(f"The number of edges in the GCC (WCC) is: {len(G.edges())}")

The number of nodes in the GCC (WCC) is: 78020
The number of edges in the GCC (WCC) is: 175130


In [55]:
G.nodes[0]

{'label': 44}

In [56]:
nx.write_graphml(G, "FirmGraphUNweightedSctrLabled.graphml")